In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Parag Jadhav\\Downloads\\ML_Deployment\\red_wine_qualit_prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Parag Jadhav\\Downloads\\ML_Deployment\\red_wine_qualit_prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
#   alpha: float
#    l1_ratio: float
    n_estimators: int 
    min_samples_split: int
    max_features: float
    max_depth: int
    criterion: str
    target_column: str

In [6]:
from src.mlproject.constant import *
from src.mlproject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
   #     params = self.params.ElasticNet
        params = self.params.RandomForestRegressor
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
 #           alpha = params.alpha,
  #          l1_ratio = params.l1_ratio,

            n_estimators = params.n_estimators, 
            min_samples_split = params.min_samples_split,
            max_features = params.max_features,
            max_depth = params.max_depth,
            criterion = params.criterion,

            target_column = schema.name
        )

        return model_trainer_config

In [8]:
import pandas as pd
import os
from src.mlproject import logger
#from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
import joblib

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        X_train = train_data.drop([self.config.target_column], axis=1)
        y_train = train_data[[self.config.target_column]]
        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[[self.config.target_column]]


#        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
#       lr.fit(train_x, train_y)

        model = RandomForestRegressor(n_estimators = self.config.n_estimators,
                                      min_samples_split = self.config.min_samples_split,
                                      max_features = self.config.max_features,
                                      max_depth = self.config.max_depth,
                                      criterion = self.config.criterion,
                                      random_state=42)
        model.fit(X_train, y_train)

        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))

In [11]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-05-11 09:36:20,051: line_number-43: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-11 09:36:20,056: line_number-43: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-11 09:36:20,061: line_number-43: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-11 09:36:20,065: line_number-63: INFO: common: created directory at: artifacts]
[2024-05-11 09:36:20,067: line_number-63: INFO: common: created directory at: artifacts/model_trainer]


c:\Users\Parag Jadhav\Downloads\ML_Deployment\red_wine_qualit_prediction\venv\lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
